>
> # MaaSSim tutorial

 ### spatial patterns

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Polygon, Point, LineString
import pandas as pd
import geopandas as gpd
import folium
from IPython.display import Image
import selenium.webdriver
import shapely.geometry
import osmnx as ox
import pyproj
from geojson.feature import Feature, FeatureCollection
from shapely.geometry import Point, Polygon, LineString
phantom_path = '/Users/rkucharski/anaconda3/lib/node_modules/phantomjs-prebuilt/lib/phantom/bin/phantomjs'
import h3
import json

In [3]:
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
import seaborn as sns
import h3
import pandas as pd
import matplotlib.pyplot as plt
from dotmap import DotMap
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

prepare

In [4]:
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, collect_results  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate, simulate_parallel
from MaaSSim.traveller import travellerEvent
from MaaSSim.driver import driverEvent

In [5]:
params = get_config('../../data/config/amsterdam.json')  # load configuration

In [6]:
params.paths.dumps = 'hexes'
params.times.patience = 1200
params.simTime = 4
params.parallel.nThread = 4
params.parallel.nReplications = 1

In [7]:
inData = load_G(inData, params)  # load network graph 

In [8]:
space = DotMap()
space.nP = [200]
space.nV = [10]
dict(space)

{'nP': [200], 'nV': [10]}

simulate

In [9]:
simulate_parallel(params = params, search_space = space)

20
0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
16    16
17    17
18    18
19    19
Name: pax_id, dtype: int64
11517
10935


AttributeError: 'Map' object has no attribute 'showMap'

### collect

demand structure

In [ ]:
res = collect_results('hexes')

In [ ]:
def plot_hex_wait_vehs(inData, res, APERTURE_SIZE = 9, threshold = 1, col = 'wait_time'):  
    traces = res.rides[['veh','pos','t']]
    diffs = traces.diff()
    diffs.columns = [_+"_diff" for _ in diffs.columns]
    traces = pd.concat([traces, diffs], axis = 1)
    trips = traces[(traces.veh_diff==0) & (traces.pos_diff==0)][['pos','t_diff']]
    trips.columns = ['pos','wait_time']
    
    for i in range(0,len(trips['pos'])):
        if trips['pos'].iloc[i] not in inData.nodes.index:
            trips['pos'].iloc[i] = (inData.nodes.index.to_series() - trips['pos'].iloc[i]).abs().idxmin()

    trips['x'] = trips.apply(lambda row: inData.nodes.loc[row.pos].x, axis = 1)
    trips['y'] = trips.apply(lambda row: inData.nodes.loc[row.pos].y, axis = 1)
    trips['hex_o_{}'.format(APERTURE_SIZE)] = trips.apply(lambda row: h3.geo_to_h3(row.y,row.x,APERTURE_SIZE),axis = 1) 
    # return the hexagon index of each place the driver is waiting at.
    
    
    col_geom = 'hex_o_{}'.format(APERTURE_SIZE)
    hexes = pd.Series(list(set(list(trips[col_geom].unique())+list(trips[col_geom].unique())))).to_frame(col_geom)
    hexes = hexes.set_index(col_geom)
    hexes[col_geom] = hexes.index.copy()
    hexes['nobs'] = trips.groupby(col_geom).size()
    # calculate the waiting time in each grid
    hexes = hexes[hexes['nobs']>threshold]
    hexes['geom'] = hexes.apply(lambda x: {"type": "Polygon","coordinates": [h3.h3_to_geo_boundary(h = x[col_geom], geo_json = True)]}, axis = 1)
    aggr = trips.groupby(col_geom)[col]
    hexes['nobs'] = aggr.size()
    hexes[col] = aggr.mean()/60
    hexes[col+'_std'] = aggr.std()
    list_features = []
    print(hexes)
    for i, row in hexes.iterrows():
        feature = Feature(geometry = row["geom"],
                          id = row[col_geom],
                          properties = {"resolution": 9})
        list_features.append(feature)
    
    feat_collection = FeatureCollection(list_features)
    geojson_hexes = json.dumps(feat_collection)
    CENTER = list(inData.nodes.loc[inData.stats.center][['y','x']].values)
    tile = 'cartodbpositron'
    base_map = folium.Map(location=CENTER, zoom_start=13,tiles=tile, zoomControl =  False)
    bins = [0, 1, 2, 3, 5, 10, 15, 20, 25, 115]
    m = folium.Choropleth(geo_data = geojson_hexes,data = hexes, columns = [col_geom,col],key_on ="feature.id",
            fill_color='Blues', control = False, bins = bins,
            fill_opacity=0.7, line_opacity=0.1).add_to(base_map)
    return base_map
plot_hex_wait_vehs(inData, res)

                         hex_o_9  nobs  \
hex_o_9                                  
891969c9b73ffff  891969c9b73ffff    29   
89196953463ffff  89196953463ffff     7   
8919695266bffff  8919695266bffff     2   
8919695220bffff  8919695220bffff     2   
891969c9a2bffff  891969c9a2bffff     3   
...                          ...   ...   
8919695347bffff  8919695347bffff     4   
8919695328fffff  8919695328fffff     2   
891969c9b6fffff  891969c9b6fffff    14   
89196953427ffff  89196953427ffff     3   
8919695351bffff  8919695351bffff     3   

                                                              geom  wait_time  \
hex_o_9                                                                         
891969c9b73ffff  {'type': 'Polygon', 'coordinates': [((4.884221...   0.394828   
89196953463ffff  {'type': 'Polygon', 'coordinates': [((4.848925...  17.007143   
8919695266bffff  {'type': 'Polygon', 'coordinates': [((4.897625...   0.250000   
8919695220bffff  {'type': 'Polygon', 'coordinate

In [ ]:
res = collect_results("hexes")
def plot_hex_map(inData, res, APERTURE_SIZE = 9, threshold = 1, col = 'wait_time'):
    
    for i in range(0,len(res.requests['origin'])):
        if res.requests['origin'].iloc[i] not in inData.nodes.index:
            res.requests['origin'].iloc[i] = (inData.nodes.index.to_series() - res.requests['origin'].iloc[i]).abs().idxmin()
    
    res.requests['wait_time'] = res.requests.apply(lambda x: res.pax_exp[res.pax_exp['dt'] == x['dt']].loc[x.name].MEETS_DRIVER_AT_PICKUP, axis = 1)
    res.requests['x'] = res.requests.apply(lambda row: inData.nodes.loc[row.origin].x, axis = 1)
    res.requests['y'] = res.requests.apply(lambda row: inData.nodes.loc[row.origin].y, axis = 1)
    res.requests['hex_o_{}'.format(APERTURE_SIZE)] = res.requests.apply(lambda row: h3.geo_to_h3(row.y,row.x,APERTURE_SIZE),axis = 1) 
    trips = res.requests

    col_geom = 'hex_o_{}'.format(APERTURE_SIZE)
    hexes = pd.Series(list(set(list(trips[col_geom].unique())+list(trips[col_geom].unique())))).to_frame(col_geom)
    hexes = hexes.set_index(col_geom)
    hexes[col_geom] = hexes.index.copy()
    hexes['nobs'] = trips.groupby(col_geom).size()
    hexes = hexes[hexes['nobs']>threshold]
    hexes['geom'] = hexes.apply(lambda x: {"type": "Polygon","coordinates": [h3.h3_to_geo_boundary(h = x[col_geom], geo_json = True)]}, axis = 1)
    aggr = trips.groupby(col_geom)[col]
    hexes['nobs'] = aggr.size()
    hexes[col] = aggr.mean()/60
    hexes[col+'_std'] = aggr.std()
    list_features = []
    for i, row in hexes.iterrows():
        feature = Feature(geometry = row["geom"],
                          id = row[col_geom],
                          properties = {"resolution": 9})
        list_features.append(feature)

    feat_collection = FeatureCollection(list_features)
    geojson_hexes = json.dumps(feat_collection)
    CENTER = list(inData.nodes.loc[inData.stats.center][['y','x']].values)
    tile = 'cartodbpositron'
    base_map = folium.Map(location=CENTER, zoom_start=13,tiles=tile, zoomControl =  False)
    bins = [0, 1, 2, 3, 5, 10, 15, 20, 25, 167]

    m = folium.Choropleth(geo_data = geojson_hexes,data = hexes, columns = [col_geom,col],key_on ="feature.id",
            fill_color='Blues', control = False, bins = bins,
            fill_opacity=0.7, line_opacity=0.1).add_to(base_map)
    return base_map
    
plot_hex_map(inData, res, APERTURE_SIZE = 9)

-------------
Rafal Kucharski, 2020, Delft, r.m.kucharski@tudelft.nl